In [1]:
import numpy as np
import seaborn as sns
from kerastuner import Hyperband, Objective
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.python.keras.metrics import CategoricalAccuracy
from tensorflow.python.keras.optimizer_v2.adam import Adam
from tqdm.keras import TqdmCallback

from src.data.ascad import AscadRandomKey
from src.dlla.hw import fetch_traces, NUM_CLASSES, plot_predictions, plot_gradient

sns.set_style('whitegrid')

In [2]:
x_prof, y_prof, x_att, y_att = fetch_traces(AscadRandomKey().default)

### Model creation

With model hyperparameters to be optimized.

In [3]:
def build_model(hp):
    model = Sequential()

    model.add(Dense(units=hp.Int('units_0',
                    min_value=32,
                    max_value=512,
                    step=32),
                    activation='relu',
                    input_shape=(1400,)))

    for i in range(hp.Int('num_layers', 0, 4)):
        model.add(Dense(units=hp.Int(f'units_{i + 1}',
                        min_value=32,
                        max_value=512,
                        step=32),
                        activation='relu'))
    model.add(Dense(NUM_CLASSES, activation='softmax'))
    model.compile(
        optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',
        metrics=CategoricalAccuracy())

    return model

### KerasTuner Hyperparameter optimization

Using the Hyperband optimization algorithm

In [4]:
tuner = Hyperband(build_model,
    objective=Objective('val_categorical_accuracy', 'max'),
    directory='.cache/hw_tuner',
    project_name='4',
    max_epochs=20)

In [5]:
tuner.search(x=x_prof, y=y_prof, validation_data=(x_att, y_att))


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
units_0           |96                |?                 
num_layers        |3                 |?                 
learning_rate     |0.001             |?                 
tuner/epochs      |3                 |?                 
tuner/initial_e...|0                 |?                 
tuner/bracket     |2                 |?                 
tuner/round       |0                 |?                 

Epoch 1/3
6250/6250 [==============================] - 8s 1ms/step - loss: 1.7881 - categorical_accuracy: 0.2639 - val_loss: 1.7764 - val_categorical_accuracy: 0.2547

KeyboardInterrupt: 

In [ ]:
best_model = tuner.get_best_models(1)[0]

In [ ]:
y_prof.shape

In [ ]:
tuner.search_space_summary()

### HW Prediction

Predict the hamming weight by taking the weighted mean for the predicted probabilities for each class.

#### Example: Some prediction

Probabilities for each class:

In [ ]:
best_model.predict(x_att[:1])[0]

**Predicted hamming weight label**, calculated by taking the weighted mean using the predicted probabilities.

In [ ]:
np.sum(best_model.predict(x_att[:1])[0] * range(8 + 1))

In [ ]:
def make_mlp(x, y, x_attack, y_attack, params):
    mdl = Sequential()
    mdl.add(Dense(100, activation=params['activation'], input_shape=(1400,)))
    mdl.add(Dense(100, activation=params['activation']))
    mdl.add(Dense(100, activation=params['activation']))
    mdl.add(Dense(100, activation=params['activation']))
    mdl.add(Dense(NUM_CLASSES, activation='softmax'))

    mdl.compile(optimizer=params['optimizer'], loss=params['losses'], metrics=['accuracy'])

    out = mdl.fit(x, y, shuffle=True, validation_data=(x_attack, y_attack), batch_size=params['batch_size'],
                  epochs=params['epochs'], verbose=False, callbacks=[TqdmCallback(verbose=0)])

    return out, mdl

best_model = make_mlp(x_prof, y_prof, x_att, y_att, {
    'activation':'relu',
    'optimizer': Adam(lr=0.001),
    'losses': 'categorical_crossentropy',
    'batch_size': 150,
    'epochs': 5
})[1]

### Plot prediction distribution for all traces.

In [ ]:
plot_predictions(best_model, x_att, y_att)

### Plot p-gradient.

In [ ]:
plot_gradient(best_model, x_att, y_att)

